In [1]:
import os
import chardet
import re
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from textwrap import wrap

/home/andresmtr/miniconda3/envs/AndTextG3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Install

* https://docs.nvidia.com/cuda/cuda-installation-guide-linux/
* https://developer.nvidia.com/cuda-downloads
* sudo apt install nvidia-cuda-toolkit

* sudo apt install build-essential
* pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121
* pip install --no-cache-dir llama-cpp-python==0.2.77 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124
* conda install pytorch==2.4.0 -c pytorch -c nvidia
* pip install huggingface-hub

In [2]:
# Activar CUDA Launch Blocking para depuración
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
# Configurar el dispositivo para usar GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Cargar el modelo y el tokenizer de Llama 3.2
model_id = "facebook/bart-large-cnn"
pipe = pipeline(
    "summarization",
    model=model_id,
    device=0 if torch.cuda.is_available() else -1  # Usar GPU si está disponible
)

/home/andresmtr/miniconda3/envs/AndTextG3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# Ruta base donde se guardarán las noticias
base_path = '/mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/en/summaries/'

if os.path.exists(base_path):
    for archivo in os.listdir(base_path):
        ruta_archivo = os.path.join(base_path, archivo)
        if os.path.isfile(ruta_archivo):
            os.remove(ruta_archivo)
            print(f"Archivo eliminado: {ruta_archivo}")
    print("Todos los archivos han sido eliminados.")
else:
    os.makedirs(base_path)
    print("La carpeta ha sido creada.")

Archivo eliminado: /mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/en/summaries/1.txt
Archivo eliminado: /mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/en/summaries/2.txt
Archivo eliminado: /mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/en/summaries/4.txt
Archivo eliminado: /mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/en/summaries/5.txt
Archivo eliminado: /mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/en/summaries/6.txt
Archivo eliminado: /mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/en/summaries/7.txt
Archivo eliminado: /mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/en/summaries/8.txt
Archivo eliminado: /mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/en/summaries/9.txt
Todos los archivos han sido eliminados.


In [6]:
# %%
def process_text(document, number_file):
    try:
        # Leer el archivo de texto
        with open(document, "r", encoding="utf-8") as f:
            text = f.read().strip()

        # Validar si el texto está vacío o muy corto
        if not text or len(text) < 30:
            print(f"El archivo {document} está vacío o tiene muy poco texto. Omitiendo...")
            return

        # Dividir textos largos en partes más pequeñas
        max_input_length = 1024
        if len(text) > max_input_length:
            parts = [text[i:i+max_input_length] for i in range(0, len(text), max_input_length)]
            resumen = []
            for part in parts:
                result = pipe(part, max_length=130, min_length=30, do_sample=False)
                resumen.append(result[0]["summary_text"])
            generated_text = " ".join(resumen)
        else:
            resumen = pipe(text, max_length=130, min_length=30, do_sample=False)
            generated_text = resumen[0]["summary_text"]

        # Guardar el texto en un archivo
        output_path = os.path.join(base_path, f"{number_file}.txt")
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(generated_text)
        print(f"Texto generado y guardado correctamente en {output_path}.")

    except Exception as e:
        print(f"Error al procesar el archivo {document}: {e}")

In [7]:
# Ruta de la carpeta
folder_path = '/mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/en/draft/'

# Lista todos los archivos .txt en la carpeta
files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

# Ordena los archivos si es necesario
files.sort()

In [8]:
# Ciclo para procesar cada archivo
for i, file_name in enumerate(files, start=1):
    file_path = os.path.join(folder_path, file_name)
    process_text(document=file_path, number_file=i)
    # Liberar memoria GPU
    torch.cuda.empty_cache()

Texto generado y guardado correctamente en /mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/en/summaries/1.txt.
Texto generado y guardado correctamente en /mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/en/summaries/2.txt.


Your max_length is set to 130, but your input_length is only 32. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)


Texto generado y guardado correctamente en /mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/en/summaries/3.txt.


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Texto generado y guardado correctamente en /mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/en/summaries/4.txt.
Texto generado y guardado correctamente en /mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/en/summaries/5.txt.


...

Texto generado y guardado correctamente en /mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/en/summaries/7.txt.


Your max_length is set to 130, but your input_length is only 31. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)


Texto generado y guardado correctamente en /mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/en/summaries/8.txt.


Your max_length is set to 130, but your input_length is only 96. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=48)


Texto generado y guardado correctamente en /mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/en/summaries/9.txt.
